In [51]:

# Import necessary packages.
import numpy as np
import pandas as pd
import torch
import os
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
# "ConcatDataset" and "Subset" are possibly useful when doing semi-supervised learning.
from torch.utils.data import ConcatDataset, DataLoader, Subset, Dataset
from torchvision.datasets import DatasetFolder, VisionDataset

# This is for the progress bar.
from tqdm.auto import tqdm
import random
import torchvision.models as models





### 处理数据label

In [52]:
# 处理train_csv的label
train_df = pd.read_csv('./data/train.csv')
unique_labels = train_df['label'].unique()

# 创建映射字典
label_to_number = {label: idx + 1 for idx, label in enumerate(unique_labels)}

# 打印映射结果
print("Label to number mapping:", label_to_number)

# 应用映射字典
train_df['label_encoded'] = train_df['label'].map(label_to_number)

# 加1是为了从1开始编号
train_array = train_df.to_numpy()
print(train_array.shape)


Label to number mapping: {'maclura_pomifera': 1, 'ulmus_rubra': 2, 'broussonettia_papyrifera': 3, 'prunus_virginiana': 4, 'acer_rubrum': 5, 'cryptomeria_japonica': 6, 'staphylea_trifolia': 7, 'asimina_triloba': 8, 'diospyros_virginiana': 9, 'tilia_cordata': 10, 'ulmus_pumila': 11, 'quercus_muehlenbergii': 12, 'juglans_cinerea': 13, 'cercis_canadensis': 14, 'ptelea_trifoliata': 15, 'acer_palmatum': 16, 'catalpa_speciosa': 17, 'abies_concolor': 18, 'eucommia_ulmoides': 19, 'quercus_montana': 20, 'koelreuteria_paniculata': 21, 'liriodendron_tulipifera': 22, 'styrax_japonica': 23, 'malus_pumila': 24, 'prunus_sargentii': 25, 'cornus_mas': 26, 'magnolia_virginiana': 27, 'ostrya_virginiana': 28, 'magnolia_acuminata': 29, 'ilex_opaca': 30, 'acer_negundo': 31, 'fraxinus_nigra': 32, 'pyrus_calleryana': 33, 'picea_abies': 34, 'chionanthus_virginicus': 35, 'carpinus_caroliniana': 36, 'zelkova_serrata': 37, 'aesculus_pavi': 38, 'taxodium_distichum': 39, 'carya_tomentosa': 40, 'picea_pungens': 41, '

### DataSet&Transform

In [53]:

train_tfm = transforms.Compose([
    # Resize the image into a fixed shape (height = width = 128)
    transforms.Resize((224, 224)),
    # You may add some transforms here.
    # ToTensor() should be the last one of the transforms.
    transforms.ToTensor(),
])

test_tfm = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])


# 两种形式，一种传file，一种直接传Tensor
# 验证集使用K折验证

# 存图片类型，getItem时，才转为Tensor
class LeavesFileDataset(Dataset):
    def __init__(self, path, train_array, tfm=test_tfm):
        super(LeavesFileDataset).__init__()
        self.path = path
        self.train_array = train_array
        self.transform = tfm

    def __len__(self):
        return len(self.train_array)

    def __getitem__(self, idx):
        data = self.train_array[idx]

        data_ = self.path + data[0]
    
        im = Image.open(data_)
        im = self.transform(im)
        # print(im.shape)
        #im = self.data[idx]
        try:
            label = data[2]
        except:
            # print('test no label')
            label = -1  # test has no label
        return im, label








In [54]:
# AlexNet
class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet, self).__init__()
        # torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        # torch.nn.MaxPool2d(kernel_size, stride, padding)
        # input 維度 [3, 128, 128]
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 96, 11, 4, 1), 
            nn.BatchNorm2d(96),
            nn.ReLU(),
            nn.MaxPool2d(3, 2, 0),     

            nn.Conv2d(96, 256, kernel_size=5, padding = 1), 
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(3, 2, 0),     
            
            nn.Conv2d(256, 384, kernel_size=3, padding=1), nn.ReLU(),
            nn.Conv2d(384, 384, kernel_size=3, padding=1), nn.ReLU(),
            nn.Conv2d(384, 256, kernel_size=3, padding=1), nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),

            
            nn.Flatten(),
            # 这里，全连接层的输出数量是LeNet中的好几倍。使用dropout层来减轻过拟合
            nn.Linear(6400, 4096), nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(4096, 4096), nn.ReLU(),
            nn.Dropout(p=0.5),
            # 最后是输出层。由于这里使用Fashion-MNIST，所以用类别数为10，而非论文中的1000
            nn.Linear(4096, 177)
            
            
        )
       

    def forward(self, x):
        out = self.cnn(x)
        return out

### 超参数


In [55]:
patience = 6
n_epochs = 1
batch_size = 128
train_ratio = 0.95  # the ratio of data used for training, the rest will be used for validation
# training parameters
seed = 0  # random seed

_exp_name = 'Leaves'

learning_rate = 0.0001  # learning rate
model_path = './model.ckpt'  # the path where the checkpoint will be saved
device = "cuda" if torch.cuda.is_available() else "cpu"
# model parameters



In [56]:

def train(n_epochs, train_loader, valid_loader, model, criterion, optimizer, device, best_acc,stale):

    # These are used to record information in training.


    for epoch in range(n_epochs):
        model.train()
        print('best_acc:' + str(best_acc))
       
        train_loss = []
        train_accs = []
        for batch in tqdm(train_loader):
            imgs, labels = batch
            # 

            labels = labels.to(device)
            # 梯度设置下 
            optimizer.zero_grad()

            logits = model(imgs.to(device))
            loss = criterion(logits, labels)

            # 反向传播
            loss.backward()

            # Clip the gradient norms for stable training. 
            grad_norm = nn.utils.clip_grad_norm_(model.parameters(), max_norm=10)

            # Update the parameters with computed gradients.
            optimizer.step()

            # Compute the accuracy for current batch.
            acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

            # Record the loss and accuracy.
            train_loss.append(loss.item())
            train_accs.append(acc)

        train_loss = sum(train_loss) / len(train_loss)
        train_acc = sum(train_accs) / len(train_accs)

        # Print the information.
        print(f"[ Train | {epoch + 1:03d}/{n_epochs:03d} ] loss = {train_loss:.5f}, acc = {train_acc:.5f}")

        # ---------- Validation ----------
        # Make sure the model is in eval mode so that some modules like dropout are disabled and work normally.
        model.eval()

        # These are used to record information in validation.
        valid_loss = []
        valid_accs = []

        # Iterate the validation set by batches.
        for batch in tqdm(valid_loader):
            # A batch consists of image data and corresponding labels.
            imgs, labels = batch
            #imgs = imgs.half()

            # We don't need gradient in validation.
            # Using torch.no_grad() accelerates the forward process.
            with torch.no_grad():
                logits = model(imgs.to(device))

            # We can still compute the loss (but not the gradient).
            loss = criterion(logits, labels.to(device))

            # Compute the accuracy for current batch.
            acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

            # Record the loss and accuracy.
            valid_loss.append(loss.item())
            valid_accs.append(acc)
        #break

        # The average loss and accuracy for entire validation set is the average of the recorded values.
        valid_loss = sum(valid_loss) / len(valid_loss)
        valid_acc = sum(valid_accs) / len(valid_accs)

        # Print the information.
        print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")

        # update logs
        if valid_acc > best_acc:
            with open(f"./{_exp_name}_log.txt", "a"):
                print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f} -> best")
        else:
            with open(f"./{_exp_name}_log.txt", "a"):
                print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")

        # save models
        if valid_acc > best_acc:
            print(f"Best model found at epoch {epoch}, saving model")
            torch.save(model.state_dict(), f"{_exp_name}_best.ckpt")  # only save best to prevent output memory exceed error
            best_acc = valid_acc
            stale = 0
        else:
            stale += 1
            if stale > patience:
                print(f"No improvment {patience} consecutive epochs, early stopping")
                break
                
    return best_acc, stale







In [57]:

_dataset_dir = ".\\data\\"
# Construct datasets.
# The argument "loader" tells how torchvision reads the data.
print(len(train_array))
train_set = LeavesFileDataset(os.path.join(_dataset_dir), train_array,tfm=train_tfm)
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)




def k_fold_split(dataset, k_folds=5):
    indices = list(range(len(dataset)))
    np.random.shuffle(indices)
    fold_size = len(dataset) // k_folds
    folds = [indices[i * fold_size:(i + 1) * fold_size] for i in range(k_folds)]
    return folds



18353


### K折交叉验证

In [58]:


best_acc = 0
k_folds = 4
stale = 0
print('train length:'+ str(len(train_set)))
folds = k_fold_split(train_set, k_folds)
print('folds:'+ str(len(folds)))



model = AlexNet()
model.load_state_dict(torch.load(f"{_exp_name}_best.ckpt"))
# model = ...  # 初始化模型
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5) 

for fold in range(k_folds):
    print(f'FOLD {fold}')
    print('--------------------------------')


    # 根据索引创建子集
    train_ids = [idx for f in folds[:fold] + folds[fold+1:] for idx in f]
    valid_ids = folds[fold]

    train_subsampler = Subset(train_set, train_ids)
    print('train length:' + str( len(train_subsampler)))
    valid_subsampler = Subset(train_set, valid_ids)
    print('valid length:' + str( len(valid_subsampler)))

    # 创建 DataLoader
    train_loader = DataLoader(train_subsampler, batch_size=batch_size, shuffle=True)
    valid_loader = DataLoader(valid_subsampler, batch_size=batch_size, shuffle=False)

    # 调用你的训练函数

    best_acc,stale = train(n_epochs, train_loader, valid_loader, model, criterion, optimizer, device,best_acc,stale)

train length:18353
folds:4
FOLD 0
--------------------------------
train length:13764
valid length:4588
best_acc:0


  0%|          | 0/108 [00:00<?, ?it/s]

[ Train | 001/001 ] loss = 0.72697, acc = 0.76563


  0%|          | 0/36 [00:00<?, ?it/s]

[ Valid | 001/001 ] loss = 0.78422, acc = 0.75929
[ Valid | 001/001 ] loss = 0.78422, acc = 0.75929 -> best
Best model found at epoch 0, saving model
FOLD 1
--------------------------------
train length:13764
valid length:4588
best_acc:0


  0%|          | 0/108 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [34]:
# 将生成的测试结果转为

# 处理test_csv的label
test_df = pd.read_csv('./data/test.csv')
test_array = test_df.to_numpy()
print(test_array.shape)





test_set = LeavesFileDataset(os.path.join(_dataset_dir), test_array,tfm=test_tfm)
test_loader = DataLoader(test_set, batch_size=64, shuffle=False, num_workers=0, pin_memory=True)



model_best = AlexNet().to(device)
model_best.load_state_dict(torch.load(f"{_exp_name}_best.ckpt"))
model_best.eval()
prediction = []
i = 0
with torch.no_grad():
     for data in test_loader:
        # print(data.shape)
        images, labels = data
        print(++i)
        test_pred = model_best(images.to(device))
        test_label = np.argmax(test_pred.cpu().data.numpy(), axis=1)
        prediction += test_label.squeeze().tolist()

print(prediction)
print(len(prediction))


df = pd.DataFrame()
df["image"] = test_array[:,0]
df["label"] = prediction
number_to_label = {v: k for k, v in label_to_number.items()}
df["label"] = df["label"].map(number_to_label)
print(df)
df.to_csv("submission.csv",index = False)




 







(8800, 1)
torch.Size([3, 224, 224])
test no label
torch.Size([3, 224, 224])
test no label
torch.Size([3, 224, 224])
test no label
torch.Size([3, 224, 224])
test no label
torch.Size([3, 224, 224])
test no label
torch.Size([3, 224, 224])
test no label
torch.Size([3, 224, 224])
test no label
torch.Size([3, 224, 224])
test no label
torch.Size([3, 224, 224])
test no label
torch.Size([3, 224, 224])
test no label
torch.Size([3, 224, 224])
test no label
torch.Size([3, 224, 224])
test no label
torch.Size([3, 224, 224])
test no label
torch.Size([3, 224, 224])
test no label
torch.Size([3, 224, 224])
test no label
torch.Size([3, 224, 224])
test no label
torch.Size([3, 224, 224])
test no label
torch.Size([3, 224, 224])
test no label
torch.Size([3, 224, 224])
test no label
torch.Size([3, 224, 224])
test no label
torch.Size([3, 224, 224])
test no label
torch.Size([3, 224, 224])
test no label
torch.Size([3, 224, 224])
test no label
torch.Size([3, 224, 224])
test no label
torch.Size([3, 224, 224])
test